# Bedrock AgentCore Browser toolkit with LangGraph React Agent

This notebook demonstrates how to use the [Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) based browser toolkit with a LangGraph React Agent to perform automated web browsing tasks. This toolkit provides a set of tools for navigating websites, interacting with web elements, and extracting information.

## Setup and Installation

First, make sure you have the required packages installed:

In [ ]:
%pip install -q langgraph langchain 'langchain-aws[tools]'

## Import Required Libraries

In [4]:
import logging

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_aws.tools import create_browser_toolkit

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Create Browser Toolkit

In [6]:
# The toolkit handles browser session management based on thread IDs
toolkit, browser_tools = create_browser_toolkit(region="us-west-2")

# Display available tools
print(f"Available browser tools: {[tool.name for tool in browser_tools]}")

Available browser tools: ['navigate_browser', 'click_element', 'navigate_back', 'extract_text', 'extract_hyperlinks', 'get_elements', 'current_webpage', 'type_text', 'take_screenshot', 'scroll_page', 'wait_for_element']


## Initialize LLM

Set up the language model that will power our agent. We'll use Claude 3.5 Haiku through Bedrock.

In [ ]:
# Initialize the language model using bedrock_converse provider
provider = "bedrock_converse"
model_id = "us.anthropic.claude-opus-4-5-20251101-v1:0"
model_with_provider = f"{provider}:{model_id}"

# Create the model instance
model = init_chat_model(model_with_provider)

## Create React Agent

Now we'll create a React agent using LangGraph's prebuilt agent. The React agent uses a reasoning and acting approach to solve tasks step by step.

In [11]:
from langchain.agents import create_agent

agent = create_agent(
    model,
    tools=browser_tools,
    system_prompt="""You are a web browsing assistant that can navigate websites, interact with web elements, and extract information. 
Use the available browser tools to complete web browsing tasks. 
When navigating websites, first go to the URL, then explore the page content to understand its structure before performing actions.
For any search tasks, find the search box, enter the query, and submit the form.
When extracting information, be precise about what elements you want to extract.

Available tools:
- navigate_browser: Navigate to a URL
- click_element: Click on an element
- navigate_back: Go back to previous page
- extract_text: Extract text content from page
- extract_hyperlinks: Extract links from page
- get_elements: Get elements matching a selector
- current_webpage: Get current page URL and title
- type_text: Type text into input fields (forms, search boxes)
- take_screenshot: Capture a screenshot of the page
- scroll_page: Scroll the page up/down/left/right
- wait_for_element: Wait for an element to appear or reach a state

Follow these steps for each task:
1. Navigate to the required website
2. Analyze the page structure
3. Interact with elements as needed (type, click, scroll)
4. Wait for dynamic content if necessary
5. Extract and report requested information
6. Provide a clear, concise summary of findings"""
)

## Execute the Agent

Now let's run the agent on a web browsing task. We'll set up a function to execute tasks and print the results.

In [13]:
async def run_browser_agent(query: str, session_id: str = "browser_session1"):
    """
    Run the browser agent on a specific query with session tracking

    Args:
        query: The task to perform
        session_id: Unique identifier for the browser session

    Returns:
        Agent response
    """
    try:
        # Configure the session ID for thread-aware tools
        config = {"configurable": {"thread_id": session_id}}

        # Invoke the agent with the query
        result = await agent.ainvoke(
            {"messages": [{"role": "user", "content": query}]}, config=config
        )

        return result
    except Exception as e:
        logger.error(f"Error running browser agent: {e}")
        raise

## Example Tasks

Let's run some example web browsing tasks to demonstrate the agent's capabilities.

In [14]:
# Example 1: Visit a website and extract information
task1 = "Go to https://news.ycombinator.com/ and tell me the titles of the top 5 posts"

result1 = await run_browser_agent(task1)
print("Task 1 Result:")
print(result1["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:bedrock_agentcore.tools.browser_client:Starting browser session...
INFO:bedrock_agentcore.tools.browser_client:✅ Session started: 01KDX8X0CCMXREA2SC43NEMCNV
INFO:bedrock_agentcore.tools.browser_client:Generating websocket headers...
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:langchain_aws.tools.browser_session_manager:Connecting to async WebSocket endpoint for thread default: wss://bedrock-agentcore.us-west-2.amazonaws.com/browser-streams/aws.browser.v1/sessions/01KDX8X0CCMXREA2SC43NEMCNV/automation
INFO:langchain_aws.tools.browser_session_manager:Successfully connected to async browser for thread default
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 1 Result:
Based on the results, here are the **top 5 posts** on Hacker News right now:

1. **ACM Is Now Open Access**
2. **2025 Letter**
3. **OpenWorkers: Self-Hosted Cloudflare Workers in Rust**
4. **Bluetooth Headphone Jacking: A Key to Your Phone [video]**
5. **Implementing HNSW (Hierarchical Navigable Small World) Vector Search in PHP**

These are the titles of the first 5 posts currently displayed on the Hacker News front page.


In [ ]:
# Example 2: Search for information
task2 = "Go to https://www.wikipedia.org, search for 'artificial intelligence', and summarize the first paragraph"

result2 = await run_browser_agent(task2)
print("Task 2 Result:")
print(result2["messages"][-1].content)

In [16]:
# Example 3: Use type_text to fill a search form and take a screenshot
task3 = """Go to https://www.google.com, use the type_text tool to enter 'Amazon Bedrock AgentCore' 
in the search box (selector: 'textarea[name="q"]'), then take a screenshot of the page."""

result3 = await run_browser_agent(task3)
print("Task 3 Result:")
print(result3["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 3 Result:
I've completed all the tasks you requested. Here's a summary:

1. **Navigated to Google.com** - Successfully loaded the Google homepage
2. **Typed the search query** - Used the `type_text` tool to enter 'Amazon Bedrock AgentCore' into the search box using the selector `textarea[name="q"]`
3. **Captured a screenshot** - The screenshot shows the Google search page with:
   - The search term "Amazon Bedrock AgentCore" visible in the search box
   - Google's autocomplete suggestions appearing below the search box, showing related searches like:
     - "amazon bedrock agentcore"
     - "amazon bedrock agent"
     - "amazon bedrock agents"
     - And other related suggestions

The screenshot confirms that the search text was successfully entered into Google's search field.


In [ ]:
# Example 3: Scroll through a page to load more content
task3 = """Go to https://news.ycombinator.com/, scroll down by 500 pixels twice to load more content, 
then extract the titles of any additional posts you can see."""

result4 = await run_browser_agent(task4)
print("Task 4 Result:")
print(result4["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 4 Result:
I've successfully extracted all the content from Hacker News after scrolling down 1000 pixels. Here are the **30 post titles** visible on the page:

## Hacker News Posts (After Scrolling)

| # | Title | Source |
|---|-------|--------|
| 1 | ACM Is Now Open Access | acm.org |
| 2 | 2025 Letter | danwang.co |
| 3 | OpenWorkers: Self-Hosted Cloudflare Workers in Rust | openworkers.com |
| 4 | Python Numbers Every Programmer Should Know | mkennedy.codes |
| 5 | Bluetooth Headphone Jacking: A Key to Your Phone [video] | ccc.de |
| 6 | Implementing HNSW (Hierarchical Navigable Small World) Vector Search in PHP | centamori.com |
| 7 | Common Lisp SDK for the Datastar Hypermedia Framework | github.com/fsmunoz |
| 8 | Sony PS5 ROM keys leaked – jailbreaking could be made easier with BootROM codes | tomshardware.com |
| 9 | Heap Overflow in FFmpeg EXIF | pwno.io |
| 10 | iOS allows alternative browser engines in Japan | developer.apple.com |
| 11 | Build a Deep Learning Library | 

In [ ]:
# Example 4: Wait for element before interacting
task4 = """Go to https://www.wikipedia.org, wait for the search input to be visible 
(selector: '#searchInput'), then type 'machine learning' and extract the search suggestions that appear."""

result5 = await run_browser_agent(task5)
print("Task 5 Result:")
print(result5["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 5 Result:
I successfully completed the task. Here's a summary of what I found:

## Summary

I navigated to Wikipedia.org, waited for the search input to become visible, typed "machine learning", and extracted the search suggestions that appeared. 

### Search Suggestions Found:

1. **Machine learning** - Study of algorithms that improve automatically through experience
2. **Machine learning in bioinformatics** - Software for understanding biological data
3. **Machine learning in earth sciences**
4. **Machine learning in video games** - Overview of the use of machine learning in several video games
5. **Machine learning in physics** - Applications of machine learning to quantum physics
6. **Machine Learning (journal)** - Academic journal

The Wikipedia search autocomplete feature provided 6 relevant suggestions based on the query "machine learning", each with a title and some with brief descriptions explaining the topic.


## Clean Up Resources

Always clean up browser resources when done to avoid unnecessary resource usage.

In [ ]:
# Clean up browser resources
await toolkit.cleanup()
print("Browser resources cleaned up successfully")